In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, LSTM, Reshape, BatchNormalization, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2
import os


In [2]:
def load_deepwriting_data(data_path):
    images = []
    labels = []
    for label_folder in os.listdir(data_path):
        label_path = os.path.join(data_path, label_folder)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (256, 64))  # Resize to match input dimensions
                img = img.astype(np.float32) / 255.0  # Normalize
                images.append(img)
                labels.append(label_folder)  # Assuming folder name is the label
    
    images = np.array(images).reshape(-1, 64, 256, 1)
    labels = np.array(labels)
    return images, labels

In [3]:
img_width, img_height = 256, 64  # Adjust based on DeepWriting dataset
input_shape = (img_height, img_width, 1)
num_classes = 200  # Adjust based on character set


In [4]:
data_path = "D:\Coding\VS Code\AI ML\HKR\Deepwriting Dataset"
x_data, y_data = load_deepwriting_data(data_path)

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)inputs = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2,2))(x)


In [ ]:
x = Reshape(target_shape=(-1, 128))(x)

In [ ]:
x = Bidirectional(LSTM(256, return_sequences=True))(x)
x = Bidirectional(LSTM(256, return_sequences=True))(x)
x = Dense(num_classes + 1, activation='softmax')(x)


In [ ]:
model = Model(inputs, x)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=25, batch_size=32)


In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
